In [ ]:
path = '/content/drive/My Drive/Colab Notebooks/2020-PoscoICT/Data/'

In [ ]:
import numpy as np
import gensim
import pandas as pd

In [ ]:
test_df = pd.read_csv(path+'news_test.csv', sep='\t', encoding='utf-8')
test_df.columns
test_df = test_df[['Topic','News']]
test_df.head()

In [ ]:
test_df['sentences'] = test_df['News'].apply(lambda x: [y.replace('<p>','').replace('</p>','').strip() for y in x.split("</p> <p>")])
test_df.head()

In [ ]:
model = gensim.models.Word2Vec.load(path+'ko/ko.bin') # 모델 로드
model.vector_size

In [ ]:
model_dict = dict()
for vocab in model.wv.vocab:
  model_dict[vocab] = model[vocab]

In [ ]:
model_dict.keys()

In [ ]:
model['도입부']

In [ ]:
embedding_dim = 200
zero_vector = np.zeros(embedding_dim)

In [ ]:
# 단어 벡터의 평균으로부터 문장 벡터를 얻는다.
def calculate_sentence_vector(sentence):
  if len(sentence) != 0:
    return sum([model_dict.get(word, zero_vector) for word in sentence])/len(sentence)
  else:
    return zero_vector

In [ ]:
# 각 문장에 대해서 문장 벡터를 반환
def sentences_to_vectors(sentences):
    return [calculate_sentence_vector(sentence) for sentence in sentences]

In [ ]:
import numpy as np
import re
import pandas as pd
import matplotlib.pyplot as plt
from nltk.tokenize import sent_tokenize, word_tokenize
from nltk.corpus import stopwords
from urllib.request import urlretrieve
import zipfile
from sklearn.metrics.pairwise import cosine_similarity
import networkx as nx

In [ ]:
test_df['sentence_embeddings'] = test_df['sentences'].apply(sentences_to_vectors)

In [ ]:
test_df.head()

In [ ]:
def similarity_matrix(sentence_embedding):
  sim_mat = np.zeros([len(sentence_embedding), len(sentence_embedding)])
  for i in range(len(sentence_embedding)):
      for j in range(len(sentence_embedding)):
        sim_mat[i][j] = cosine_similarity(sentence_embedding[i].reshape(1, embedding_dim),
                                          sentence_embedding[j].reshape(1, embedding_dim))[0,0]
  return sim_mat

In [ ]:
test_df['SimMatrix'] = test_df['sentence_embeddings'].apply(similarity_matrix)


In [ ]:
test_df.head()

In [ ]:
def calculate_score(sim_matrix):
    nx_graph = nx.from_numpy_array(sim_matrix)
    scores = nx.pagerank(nx_graph)
    return scores

In [ ]:
test_df['score'] = test_df['SimMatrix'].apply(calculate_score)
test_df[['SimMatrix', 'score']]

In [29]:
def ranked_sentences(sentences, scores, n=3):
    top_scores = sorted(((scores[i],s) for i,s in enumerate(sentences)), reverse=True)
    top_n_sentences = [sentence for score,sentence in top_scores[:n]]
    return '\n'.join(top_n_sentences)

In [30]:
test_df['summary'] = test_df.apply(lambda x: ranked_sentences(x.sentences, x.score), axis=1)

In [31]:
for i in range(0, 5):
  print("=====")
  print(i+1,'document')
  print('원문 :',test_df.loc[i].News)
  print('요약 :',test_df.loc[i].summary)

=====
1 document
원문 : <p> [월드 IT쇼 2009] 다채로운 부대행사‥해외 바이어와 1대 1 수출 상담 </p> <p> '월드IT쇼(WIS)2009' 부대행사도 다채롭게 열린다. 국내 정보기술(IT) 기업들의 해외시장 개척을 지원해주는 수출상담회는 물론 미국 조달시장에 대한 한국 IT기업의 진출을 돕기 위한 행사도 따로 마련된다. 대학에서 개발한 신기술을 산 · 학 협력으로 연결해주는 자리도 있다. </p> <p> IT분야 중소기업의 세계시장 판로 개척을 지원하는 다양한 수출상담회가 풍성하게 열린다. 특히 기술 및 제품 구매를 직접 관장하는 해외 바이어와 국내 기업 간 일대일 비즈니스 상담 방식으로 이뤄져 실질적인 성과가 클 것으로 기대를 모으고 있다. </p> <p> 18일 오전 10시부터 오후 5시까지 코엑스에서 '해외 바이어 초청 수출상담회'가 열린다. 월드IT쇼에 참가하는 해외 바이어들을 국내 기업과 연결해주는 행사다. 유럽 최대 전자제품 유통기업인 미디어마켓 그룹의 구매담당자가 직접 방한해 한국 제품을 구매할 예정이다. </p> <p> 17일과 18일 양일간에 걸쳐 미국 </p> <p> 버지니아주 페어팩스 카운티의 미국 조달시장 세미나 및 수출상담회도 열린다. </p> <p> 19일에는 미국 조지아주의 IT포럼도 미국시장 진출을 원하는 기업들에 도움이 될 전망이다. </p> <p> 버지니아주 페어팩스 카운티가 주최하는 미국 IT시장 및 조달시장 진출 전략 세미나는 17일 오후 2시 코엑스 3층 홀E 332호에서 열린다. 일대일 비즈니스 미팅 형식의 상담도 진행된다. 이 세미나에서는 미국 IT시장 및 조달제도와 관련해 페어팩스 카운티의 주요 산업 분야인 국방,보안,통신,그린 에너지 관련 분야 조달 제도 및 시장 정보가 상세하게 소개된다. 18일에는 일대일 비즈니스 미팅이 이뤄지고 아시아 지역 비즈니스 담당관이 직접 참석해 국내 기업들과 상담한다. </p> <p> 19일에는 미국 조지아주 정부가 '세계로 뻗어나가는 조지아주의 IT산업'